In [32]:
import numpy as np
import tqdm

In [42]:
OD = np.random.randint(100,10000, (1000,1000))
thresh = 200

In [43]:
# initialize the population vector from the origin-destination flow matrix
N_k = np.abs(np.diagonal(OD) + OD.sum(axis=0) - OD.sum(axis=1))
locs_len = len(N_k)                 # number of locations
SIR = np.zeros(shape=(locs_len, 3)) # make a numpy array with 3 columns for keeping track of the S, I, R groups
SIR[:,0] = N_k                      # initialize the S group with the respective populations

In [44]:
first_infections = np.where(SIR[:, 0]<=thresh, SIR[:, 0]//20, 0)   # for demo purposes, randomly introduce infections
SIR[:, 0] = SIR[:, 0] - first_infections
SIR[:, 1] = SIR[:, 1] + first_infections                           # move infections to the I group

In [45]:
# row normalize the SIR matrix for keeping track of group proportions
row_sums = SIR.sum(axis=1)
SIR_n = SIR / row_sums[:, np.newaxis]

In [46]:
# initialize parameters
beta = 1.6
gamma = 0.04
public_trans = 0.5                                 # alpha
R0 = beta/gamma
beta_vec = np.random.gamma(1.6, 2, locs_len)
gamma_vec = np.full(locs_len, gamma)
public_trans_vec = np.full(locs_len, public_trans)

In [47]:
# make copy of the SIR matrices 
SIR_sim = SIR.copy()
SIR_nsim = SIR_n.copy()

In [48]:
# run model
print(SIR_sim.sum(axis=0).sum() == N_k.sum())
infected_pop_norm = []
susceptible_pop_norm = []
recovered_pop_norm = []
for time_step in tqdm.notebook.tqdm(range(100)):
  infected_mat = np.array([SIR_nsim[:,1],]*locs_len).transpose()
  OD_infected = np.round(OD*infected_mat)
  inflow_infected = OD_infected.sum(axis=0)
  inflow_infected = np.round(inflow_infected*public_trans_vec)
  print('total infected inflow: ', inflow_infected.sum())
  new_infect = beta_vec*SIR_sim[:, 0]*inflow_infected/(N_k + OD.sum(axis=0))
  new_recovered = gamma_vec*SIR_sim[:, 1]
  new_infect = np.where(new_infect>SIR_sim[:, 0], SIR_sim[:, 0], new_infect)
  SIR_sim[:, 0] = SIR_sim[:, 0] - new_infect
  SIR_sim[:, 1] = SIR_sim[:, 1] + new_infect - new_recovered
  SIR_sim[:, 2] = SIR_sim[:, 2] + new_recovered
  SIR_sim = np.where(SIR_sim<0,0,SIR_sim)
  # recompute the normalized SIR matrix
  row_sums = SIR_sim.sum(axis=1)
  SIR_nsim = SIR_sim / row_sums[:, np.newaxis]
  S = SIR_sim[:,0].sum()/N_k.sum()
  I = SIR_sim[:,1].sum()/N_k.sum()
  R = SIR_sim[:,2].sum()/N_k.sum()
  print(S, I, R, (S+I+R)*N_k.sum(), N_k.sum())
  print('\n')
  infected_pop_norm.append(I)
  susceptible_pop_norm.append(S)
  recovered_pop_norm.append(R)

True


total infected inflow:  89885.0
0.999945019616629 5.497998551764964e-05 3.978534335203871e-10 100539537.0 100539537


total infected inflow:  188193.0
0.9998302485479023 0.00016755185482353438 2.1995972741395065e-06 100539536.99999999 100539537


total infected inflow:  496172.0
0.9995277122873358 0.00046338604119715337 8.901671467080881e-06 100539537.0 100539537


total infected inflow:  1280675.0
0.998747964621047 0.0012245982658381322 2.743711311496702e-05 100539537.0 100539537


total infected inflow:  3267040.0
0.9967615666421127 0.0031620123141387537 7.642104374849232e-05 100539536.99999999 100539537


total infected inflow:  8311111.0
0.9917247796583761 0.008072318805309838 0.0002029015363140425 100539537.0 100539537


total infected inflow:  21091786.0
0.979046754091821 0.020427451619652714 0.000525794288526436 100539537.00000001 100539537


total infected inflow:  53244400.0
0.947700823145837 0.050956284500850324 0.0013428923533125446 100539536.99999999 100539537


total infec

0.004523343193237887 0.09866126342911506 0.8968153933776473 100539537.00000001 100539537


total infected inflow:  248426926.0
0.004484585169642645 0.09475357091554573 0.9007618439148116 100539537.0 100539537


total infected inflow:  238587672.0
0.0044477630304734575 0.09100025021809308 0.9045519867514334 100539537.0 100539537


total infected inflow:  229136978.0
0.004412764199740374 0.08739523904010243 0.9081919967601572 100539537.0 100539537


total infected inflow:  220059314.0
0.004379483707846203 0.0839327099703925 0.9116878063217614 100539537.0 100539537


total infected inflow:  211340685.0
0.004347824260020801 0.08060706101940221 0.9150451147205769 100539536.99999999 100539537


total infected inflow:  202966903.0
0.004317694771654203 0.07741290806699272 0.9182693971613531 100539537.0 100539537


total infected inflow:  194924317.0
0.004289010633170585 0.07434507588279662 0.9213659134840327 100539537.0 100539537


total infected inflow:  187199702.0
0.004261692374285878 0.071